![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Model Editing  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/mjspec.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for model editing in MuJoCo using the `mjSpec` API. This notebook assumes that the reader is already familiar with MuJoCo basic concepts, as demostrated in the [introductory tutorial](https://github.com/google-deepmind/mujoco?tab=readme-ov-file#getting-started). Documentation for this API can be found in the [Model Editing](https://mujoco.readthedocs.io/en/latest/programming/modeledit.html) chapter in the documentation (C API) and in the [Python chapter](https://mujoco.readthedocs.io/en/latest/python.html#model-editing). Here we use the Python API.

The goal of the API is to allow users to easily interact with and modify MuJoCo
models in Python, similarly to what the JavaScript DOM does for HTML.

<!-- Copyright 2024 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->


## All imports

In [ ]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco as mj
  mj.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np
import typing
import dataclasses
import random
import math

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

from pygments import highlight
from pygments.lexers import XmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display

def print_xml(xml_string):
    formatter = HtmlFormatter(style='lovelace')
    highlighted_xml = highlight(xml_string, XmlLexer(), formatter)
    display(HTML(f"<style>{formatter.get_style_defs()}</style>{highlighted_xml}"))

import matplotlib.pyplot as plt
import matplotlib.animation as animation

def render(model, data=None, height=250):
  if data is None:
    data = mj.MjData(model)
  with mj.Renderer(model, 480, 640) as renderer:
    mj.mj_forward(model, data)
    renderer.update_scene(data)
    media.show_image(renderer.render(), height=height)


# Parsing XML to `mjSpec` and compiling to `mjModel`


Unlike `mj_loadXML` which combines parsing and compiling, when using `mjSpec`, parsing and compiling are separate, allowing for editing steps:

In [ ]:
#@title A static model, from string {vertical-output: true}

static_model = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
spec = mj.MjSpec.from_string(static_model)
model = spec.compile()
render(model)

# Change the mjSpec, re-compile and re-render
geoms = spec.worldbody.find_all(mj.mjtObj.mjOBJ_GEOM)
geoms[0].name = 'blue_box'
geoms[0].rgba = [0, 0, 1, 1]
geoms[1].name = 'yellow_sphere'
geoms[1].rgba = [1, 1, 0, 1]
spec.worldbody.add_geom(name='magenta cylinder',
                        type=mj.mjtGeom.mjGEOM_CYLINDER,
                        rgba=[1, 0, 1, 1],
                        pos=[-.2, 0, .2],
                        size=[.1, .1, 0])

model = spec.compile()
render(model)

`mjSpec` can save XML to string, saving all modifications.

In [ ]:
#@title Print an XML from an `mjSpec` {vertical-output: true}

print_xml(spec.to_xml())

In [ ]:
#@title Building an `mjSpec` from scratch {vertical-output: true}

spec = mj.MjSpec()
spec.worldbody.add_light(name="top", pos=[0, 0, 1])
body = spec.worldbody.add_body(name="box_and_sphere", euler=[0, 0, -30])
body.add_joint(name="swing", type=mj.mjtJoint.mjJNT_HINGE,
               axis=[1, -1, 0], pos=[-.2, -.2, -.2])
body.add_geom(name="red_box", type=mj.mjtGeom.mjGEOM_BOX,
              size=[.2, .2, .2], rgba=[1, 0, 0, 1])
body.add_geom(name="green_sphere", pos=[.2, .2, .2],
              size=[.1, 0, 0], rgba=[0, 1, 0, 1])
model = spec.compile()

duration = 2    # (seconds)
framerate = 30  # (Hz)

# enable joint visualization option:
scene_option = mj.MjvOption()
scene_option.flags[mj.mjtVisFlag.mjVIS_JOINT] = True

# Simulate and display video.
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)


# Control example

A key feature of this library is the ability to easily attach multiple models into a larger one. Disambiguation of duplicated names from different
models, or multiple instances of the same model, is handled via user-defined namespacing.

One example use case is when we want robots with a variable number of joints, as is a fundamental change to the kinematic structure. The following snippets realise this scenario.

In [ ]:
leg_model = """
<mujoco>
  <compiler angle="radian"/>

  <default>
    <joint damping="2" type="hinge"/>
    <geom type="capsule"/>
  </default>

  <worldbody>
    <body name="thigh">
      <joint name="hip" axis="0 0 1"/>
      <body name="shin">
        <joint name="knee" axis="0 1 0"/>
      </body>
    </body>
  </worldbody>

  <actuator>
    <position joint="hip" kp="10"/>
    <position joint="knee" kp="10"/>
  </actuator>
</mujoco>
"""

class Leg(object):
  """A 2-DoF leg with position actuators."""
  def __init__(self, length, rgba):
    self.spec = mj.MjSpec.from_string(leg_model)

    # Thigh:
    thigh = self.spec.find_body('thigh')
    thigh.add_geom(fromto=[0, 0, 0, length, 0, 0], size=[length/4, 0, 0], rgba=rgba)

    # Hip:
    shin = self.spec.find_body('shin')
    shin.add_geom(fromto=[0, 0, 0, 0, 0, -length], size=[length/5, 0, 0], rgba=rgba)
    shin.pos[0] = length

The `Leg` class describes an abstract articulated leg, with two joints and corresponding proportional-derivative actuators.

Note that:

- MJCF attributes correspond directly to arguments of the `add_()` methods.
- When referencing elements, e.g when specifying the joint to which an actuator is attached, the name string of the MJCF elements is used.

In [ ]:
BODY_RADIUS = 0.1
random_state = np.random.RandomState(42)
creature_model = """
<mujoco>
  <compiler angle="radian"/>

  <worldbody>
    <geom name="torso" type="ellipsoid" size="{} {} {}"/>
  </worldbody>
</mujoco>
""".format(BODY_RADIUS, BODY_RADIUS, BODY_RADIUS / 2)

def make_creature(num_legs):
  """Constructs a creature with `num_legs` legs."""
  rgba = random_state.uniform([0, 0, 0, 1], [1, 1, 1, 1])
  spec = mj.MjSpec.from_string(creature_model)

  # Attach legs to equidistant sites on the circumference.
  spec.worldbody.first_geom().rgba = rgba
  leg = Leg(length=BODY_RADIUS, rgba=rgba)
  for i in range(num_legs):
    theta = 2 * i * np.pi / num_legs
    hip_pos = BODY_RADIUS * np.array([np.cos(theta), np.sin(theta), 0])
    hip_site = spec.worldbody.add_site(pos=hip_pos, euler=[0, 0, theta])
    hip_site.attach(leg.spec.find_body('thigh'), '', '-' + str(i))

  return spec

The `make_creature` function uses the `attach()` method to procedurally attach legs to the torso. Note that at this stage both the torso and hip attachment sites are children of the `worldbody`, since their parent body has yet to be instantiated. We'll now make an arena with a chequered floor and two lights, and place our creatures in a grid.

In [ ]:
#@title Six Creatures on a floor.{vertical-output: true}

arena = mj.MjSpec()

if hasattr(arena, 'compiler'):
  arena.compiler.degree = False  # MuJoCo dev (next release).
else:
  arena.degree = False  # MuJoCo release

# Make arena with textured floor.
chequered = arena.add_texture(
    name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
    builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
    width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
grid = arena.add_material(
    name='grid', texrepeat=[5, 5], reflectance=.2
    ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
arena.worldbody.add_geom(
    type=mj.mjtGeom.mjGEOM_PLANE, size=[2, 2, .1], material='grid')
for x in [-2, 2]:
  arena.worldbody.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])

# Instantiate 6 creatures with 3 to 8 legs.
creatures = [make_creature(num_legs=num_legs) for num_legs in range(3, 9)]

# Place them on a grid in the arena.
height = .15
grid = 5 * BODY_RADIUS
xpos, ypos, zpos = np.meshgrid([-grid, 0, grid], [0, grid], [height])
for i, spec in enumerate(creatures):
  # Place spawn sites on a grid.
  spawn_pos = (xpos.flat[i], ypos.flat[i], zpos.flat[i])
  spawn_site = arena.worldbody.add_site(pos=spawn_pos, group=3)
  # Attach to the arena at the spawn sites, with a free joint.
  spawn_body = spawn_site.attach(spec.worldbody, '', '-' + str(i))
  spawn_body.add_freejoint()

# Instantiate the physics and render.
model = arena.compile()
render(model)

Multi-legged creatures, ready to roam! Let's inject some controls and watch them move. We'll generate a sinusoidal open-loop control signal of fixed frequency and random phase, recording both video frames and the horizontal positions of the torso geoms, in order to plot the movement trajectories.

In [ ]:
#@title Video of the movement{vertical-output: true}

duration = 10   # (Seconds)
framerate = 30  # (Hz)
video = []
pos_x = []
pos_y = []
geoms = arena.worldbody.find_all(mj.mjtObj.mjOBJ_GEOM)
torsos = [geom.id for geom in geoms if 'torso' in geom.name]
actuators = [actuator.id for actuator in arena.actuators]

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.actuators))
amp = 0.9

# Simulate, saving video frames and torso locations.
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)

    # Save torso horizontal positions using name indexing.
    pos_x.append(data.geom_xpos[torsos, 0].copy())
    pos_y.append(data.geom_xpos[torsos, 1].copy())

    # Save video frames.
    if len(video) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      video.append(pixels.copy())

media.show_video(video, fps=framerate)

In [ ]:
#@title Movement trajectories{vertical-output: true}

creature_colors = model.geom_rgba[torsos][:, :3]
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_prop_cycle(color=creature_colors)
_ = ax.plot(pos_x, pos_y, linewidth=4)

The plot above shows the corresponding movement trajectories of creature positions. Note how `mjSpec` attribute `id` were used to access both `xpos` and `rgba` values. This attribute is valid only after a model is compiled.

# Model editing

First we need to download models from mujoco_menagerie

In [ ]:
# Get MuJoCo's humanoid model and a Franka arm from the MuJoCo Menagerie.
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco
humanoid_file = 'mujoco/model/humanoid/humanoid.xml'
humanoid100_file = 'mujoco/model/humanoid/humanoid100.xml'
print('Getting MuJoCo Menagerie Franka XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco_menagerie
franka_file = 'mujoco_menagerie/franka_fr3/fr3.xml'

`mjSpec` elements can be traversed in two ways:
- For elements inside the kinematic tree, the tree can be traversed using the `first` and `next` functions.
- For all other elements, we provide a list.



In [ ]:
#@title Traversing the spec.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

# Function that recursively prints all body names
def print_bodies(parent, level=0):
  body = parent.first_body()
  while body:
    print(''.join(['-' for i in range(level)]) + body.name)
    print_bodies(body, level + 1)
    body = parent.next_body(body)

print("The spec has the following actuators:")
for actuator in spec.actuators:
  print(actuator.name)

print("\nThe spec has the following bodies:")
print_bodies(spec.worldbody)

An `mjSpec` can be compiled multiple times. If the state has to be preserved between different compilations, then the function `recompile()` must be used, which returns a new `mjData` that contains the mapped state, possibly having a different dimension from the origin.

In [ ]:
#@title Model re-compilation with state preservation.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid100_file)
model = spec.compile()
data = mj.MjData(model)

# Run for 5 seconds
for i in range(1000):
  mj.mj_step(model, data)

# Show result
render(model, data)

# Create list of all bodies we want to delete
body = spec.worldbody.first_body()
delete_list = []
while body:
  geom_type = body.first_geom().type
  if (geom_type == mj.mjtGeom.mjGEOM_BOX or
      geom_type == mj.mjtGeom.mjGEOM_ELLIPSOID):
    delete_list.append(body)
  body = spec.worldbody.next_body(body)

# Remove all bodies in the list from the spec
for body in delete_list:
  spec.detach_body(body)

# # Add another humanoid
spec_humanoid = mj.MjSpec.from_file(humanoid_file)
attachment_frame = spec.worldbody.add_frame(pos=[0, -1, 2])
attachment_frame.attach_body(spec_humanoid.find_body('torso'), 'a', 'b')

# Recompile preserving the state
new_model, new_data = spec.recompile(model, data)

# Show result
render(new_model, new_data)

Let us load the humanoid model and inspect it.

In [ ]:
#@title Humanoid model.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

model = spec.compile()
render(model)

We wish to remove the arms and replace them with the legs. This can be done by first storing the arm positions into frames attaches to the torso. Then we can detach the arms and self-attach the legs into the frames.

In [ ]:
#@title Humanoid with arms replaced by legs.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

# Get the torso, arm, and leg bodies
arm_left = spec.find_body('upper_arm_left')
arm_right = spec.find_body('upper_arm_right')
leg_left = spec.find_body('thigh_left')
leg_right = spec.find_body('thigh_right')
torso = spec.find_body('torso')

# Attach frames at the arm positions
shoulder_left = torso.add_frame(pos=arm_left.pos)
shoulder_right = torso.add_frame(pos=arm_right.pos)

# Remove the arms
spec.detach_body(arm_left)
spec.detach_body(arm_right)

# Add new legs
shoulder_left.attach_body(leg_left, 'shoulder', 'left')
shoulder_right.attach_body(leg_right, 'shoulder', 'right')

model = spec.compile()
render(model, height=400)

Similarly, different models can be attach together. Here, the right arm is detached and a robot arm from a different model is attached in its place.

In [ ]:
#@title Humanoid with Franka arm.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)
franka = mj.MjSpec.from_file(franka_file)

if hasattr(spec, 'compiler'):
  spec.compiler.degree = False  # MuJoCo dev (next release).
else:
  spec.degree = False  # MuJoCo release

# Replace right arm with frame
arm_right = spec.find_body('upper_arm_right')
torso = spec.find_body('torso')
shoulder_right = torso.add_frame(pos=arm_right.pos, quat=[0, 0.8509035, 0, 0.525322])
spec.detach_body(arm_right)

# Attach Franka arm to humanoid
franka_arm = franka.find_body('fr3_link2')
shoulder_right.attach_body(franka_arm, 'franka', '')

model = spec.compile()
render(model, height=400)

When doing this, the actuators and all other objects referenced by the attached sub-tree are imported in the new model. All assets are currently imported, referenced or not.

In [ ]:
#@title Imported actuators.{vertical-output: true}

for actuator in spec.actuators:
  print(actuator.name)

Domain randomization can be performed by attaching multiple times the same spec, edited each time with a new instance of randomized parameters.

In [ ]:
#@title Humanoid with randomized heads and arm poses.{vertical-output: true}

humanoid = mj.MjSpec.from_file(humanoid_file)
spec = mj.MjSpec()

# Delete all key frames to avoid name conflicts
while humanoid.keys:
  humanoid.keys[-1].delete()

# Create a grid of humanoids by attaching humanoid to spec multiple times
for i in range(4):
  for j in range(4):
    humanoid.materials[0].rgba = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), 1]  # Randomize color
    humanoid.find_body('head').first_geom().size = [
        .18*np.random.uniform(), 0, 0]  # Randomize head size
    humanoid.find_body('upper_arm_left').quat = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), np.random.uniform()]  # Randomize left arm orientation
    humanoid.find_body('upper_arm_right').quat = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), np.random.uniform()]  # Randomize right arm orientation

    # attach randomized humanoid to parent spec
    frame = spec.worldbody.add_frame(pos=[i, j, 0])
    frame.attach_body(humanoid.find_body('torso'), str(i), str(j))

spec.worldbody.add_light(mode=mj.mjtCamLight.mjCAMLIGHT_TARGETBODYCOM,
                         targetbody='3torso3', diffuse=[.8, .8, .8],
                         specular=[0.3, 0.3, 0.3], pos=[0, -6, 4], cutoff=30)
model = spec.compile()
render(model, height=400)

# Pure Models with `mjspec`

A robot operates within an enviornment. We refer to this enviornment as Arena. To add versatility to the enviornment we should be able to add or remove things from enviorment. To add these functionalities to the Arena we will use mjspec's attach, detach and frame functionality.



## Arena

In [ ]:
class Arena:
  def __init__(self):
    self.spec = mj.MjSpec()
    self.model = self.spec.worldbody
    self.spec.compiler.degree = False
    self.scene_option = mj.MjvOption()

    # Make arena with textured floor.
    chequered = self.spec.add_texture(
        name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
        builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
        width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
    grid = self.spec.add_material(
        name='grid', texrepeat=[5, 5], reflectance=.2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
    self.model.add_geom(
        type=mj.mjtGeom.mjGEOM_PLANE, size=[2, 2, .1], material='grid')
    for x in [-2, 2]:
      self.model.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])

  def add_tracking_camera(self,body,name="eye",pos=[0,0,1],euler=[0,0,0]):
    # camera
    self.model.add_camera(name=name,
                          mode =mj.mjtCamLight.mjCAMLIGHT_TRACK ,
                          targetbody=body,
                          pos= pos,
                          euler=euler)
    return name

  def visulize_camera(self):
    self.scene_option.flags[mj.mjtVisFlag.mjVIS_CAMERA] = True

  def add_movable_asset(self,asset,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(asset.model,  prefix,'')
    body.add_freejoint()
    return body

  def add_fixed_asset(self,asset,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(asset.model,  prefix,'')
    return body

  def remove_asset(self,body):
    self.spec.detach_body(body)

In [ ]:
arena = Arena()
model = arena.spec.compile()
render(model)

## Creating a Robot
Next, we will look at how to put a robot together. We are going to create a quadruped, which has four identical legs. We break down the leg model into three parts, namely the robot tree structure, actuation, and sensors.

### Robot tree structure
we start by setting default values such as color and joint damping, which are shared throughout the entire model. Next, we proceed to define the data belonging to the tree, referred to as properties. Separating the data from the tree-like structure enables quick modifications to the model if needed. Finally, the properties are used to construct the model.

In [ ]:
class LegBase:
  def __init__(self,length=0.1):
    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody

    # material
    self.spec.add_material(name = "yellow",rgba=[1,1,0,1])

    # defaults
    main = self.spec.default()
    main.joint.damping = 2

    main.geom.material = "yellow"
    main.geom.type = mj.mjtGeom.mjGEOM_CAPSULE
    main.geom.mass = 1

    # props
    bodies = [
        {
            "body":{
                "name":"thigh"
            },
            "joint":{
                "name":"hip",
                "axis":[0,0,1],
            },
            "geom":{
                "fromto":[0]*3+[length]+[0]*2,
                "size":[length/4,0,0]
            }
        },
        {
            "body":{
                "name":"shin",
                "pos":[length,0,0]
            },
            "joint":{
                "name":"knee",
                "axis":[0, 1, 0],
            },
            "geom":{
                "fromto":[0]*5+[-length],
                "size":[length/5,0,0]
            },
            "site":{
                "name":"feet",
                "pos":[0,0,-length]
            }
        }
    ]
    body = self.model
    for prop in bodies:
      body = body.add_body(**prop['body'])
      body.add_joint(**prop['joint'])
      body.add_geom(**prop['geom'])
      if  "site" in prop:
        body.add_site(**prop['site'])

In [ ]:
leg = LegBase()
model = leg.spec.compile()
print_xml(leg.spec.to_xml())

Next lets have a look at how we can add and remove a model from arena

In [ ]:
arena = Arena()
body = arena.add_fixed_asset(leg,pos=[0,0,0.15])
model = arena.spec.compile()
render(model)

In [ ]:
arena.remove_asset(body)
model = arena.spec.compile()
render(model)

### Actuator
Here we add positon actuaotors to the model. We start by drving the LegActuated class from LegBase. We need create a position actuator from a general actuator. We achived this by setting default values of the general actuator.

In [ ]:
class LegActuation(LegBase):
  def __init__(self,length=0.1,kp=10):
    super().__init__(length)

    # defaults
    main = self.spec.default()

    # position actuator defaults
    main.actuator.trntype = mj.mjtTrn.mjTRN_JOINT
    main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
    main.actuator.biastype = mj.mjtBias.mjBIAS_AFFINE
    main.actuator.gainprm[0] =  kp
    main.actuator.biasprm[1] = -kp

    actuators =[
        {
            "name":"hip_act",
            "target":"hip"
        },
        {
            "name":"knee_act",
            "target":"knee"
        },
    ]

    for prop in actuators:
      self.spec.add_actuator(**prop)

In [ ]:
leg = LegActuation()
model = leg.spec.compile()
print_xml(leg.spec.to_xml())

### Sensors
To better understand what is going on with the robot we need to add some sensors. Mujoco support different types of sensors that can be found in: mujoco/introspect
/enums.py

In order to use a sensor we need to define sensor type, object that it operates on and object name. For example a joint position sensor will have a type `mjtSensor.mjSENS_JOINTPOS` operate on object of type `mjtObj.mjOBJ_JOINT` and a name `super_duper_unique_joint_name`


In [ ]:
class Leg(LegActuation):
  def __init__(self,length=0.1,kp=10,rgba=[1,0,0,1]):
    super().__init__(length,kp)

    # props
    sesnros = [
      # joint sensors
      {
          "name":"hip_jpos",
          "objname":"hip",
          "type":mj.mjtSensor.mjSENS_JOINTPOS,
          "objtype":mj.mjtObj.mjOBJ_JOINT

      },
      # force sensor
      {
          "name":"feet_force",
          "objname":"feet",
          "type":mj.mjtSensor.mjSENS_FORCE,
          "objtype":mj.mjtObj.mjOBJ_SITE

      },
    ]

    for prop in sesnros:
      self.spec.add_sensor(**prop)

In [ ]:
leg = Leg()
model = leg.spec.compile()
print_xml(leg.spec.to_xml())

In [ ]:
arena = Arena()
robot = Leg()
arena.add_fixed_asset(robot,[0,0,0.11])
model = arena.spec.compile()
random_state = np.random.RandomState(42)

actuators = [actuator.id for actuator in arena.spec.actuators]
sensors = dict(zip(
    [sensor.name for sensor in arena.spec.sensors],
    [sensor.id for sensor in arena.spec.sensors]
))

print(f"Sensors: {sensors}")

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 3    # (seconds)
framerate = 30  # (Hz)
frames = []
hip_jpos = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)
      hip_jpos.append(data.sensordata[sensors['_hip_jpos']])

media.show_video(frames, fps=framerate)

fig, ax = plt.subplots(figsize=(4, 2))
_ = ax.plot( range(len(hip_jpos)),hip_jpos, linewidth=1)
ax.set_ylabel('hip joint position')



### Component Reusablity
Here we illusterate component reusablity by creating a quadroped using the leg component

In [ ]:
class MultiLegRobot:
  def __init__(self,body_radius=0.1,kp=10,num_legs=4):
    length = body_radius

    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody

    # material
    self.spec.add_material(name = "yellow",rgba=[1,1,0,1])

    # props
    base = {
        "body":{
            "name":"torso"
        },
        "geom":{
          "size":[body_radius]*2+[body_radius/2],
          "material":"yellow",
          "type":mj.mjtGeom.mjGEOM_ELLIPSOID
        }
    }


    # base
    body = self.model.add_body(**base['body'])
    body.add_geom(**base['geom'])

    # legs
    for i in range(num_legs):
      theta = 2 * i * np.pi / num_legs
      hip_pos = body_radius * np.array([np.cos(theta), np.sin(theta), 0])
      frame = body.add_frame(pos = hip_pos,euler = [0,0,theta])
      leg = Leg()
      frame.attach_body(leg.model, '', '-' + str(i))

In [ ]:
robot = MultiLegRobot()
model = robot.spec.compile()
print_xml(robot.spec.to_xml())

### Closed Loop Kinematic and Contact Exclusion
Here we are going to create a hexapod to iilusterate how to use equaility constraint. The tree-like structure of the hexapod lead to closed loop kinematics. Therefore we need to replace two of the joints with equality contraint. We start by creating the LinearArm that actuates the hexapod. Here is a link to a hexapod: https://www.youtube.com/watch?v=xiECumcaEx0

In [ ]:
class LinearArmBase:
  def __init__(self,length=0.1):
    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody

    # defaults
    main = self.spec.default()
    main.joint.axis = [0,0,1]

    # material
    self.spec.add_material(name = "yellow",rgba=[1,1,0,1])
    self.spec.add_material(name = "orange",rgba=[1.0, 0.5, 0.0, 1.0])

    bodies = [
        {
            "body":{
                "name":"l1",
            },
            "joint":{
                "type": mj.mjtJoint.mjJNT_HINGE,
                "axis":[1,0,0]
            },
            "geom":{
                "fromto":[0]*5+[length],
                "material":"yellow",
                "size":[length/5,0,0],
                "type":mj.mjtGeom.mjGEOM_CAPSULE
            }
        },
        {
            "body":{
                "name":"l2",
                "pos":[0,0,length/2]
            },
            "joint":{
                "type":mj.mjtJoint.mjJNT_SLIDE,
                "range":[0,0.05],
                "name":"linear",
            },
            "geom":{
                "fromto":[0]*5+[length*2],
                "size":[length/5,0,0],
                "material":"orange",
                "type":mj.mjtGeom.mjGEOM_CAPSULE
            },
            "site":{
                "pos":[0,0,length*2],
                "name":"arm_site"

            }
        }
    ]

    body = self.model
    for prop in bodies:
      body = body.add_body(**prop['body'])
      body.add_joint(**prop['joint'])
      body.add_geom(**prop['geom'])
      if  "site" in prop:
        self.site = body.add_site(**prop['site'])

In [ ]:
class LinearArm(LinearArmBase):
  def __init__(self,length=0.1,kp=20):
    super().__init__(length)

    # defaults
    main = self.spec.default()

    # position actuator defaults
    main.actuator.trntype = mj.mjtTrn.mjTRN_JOINT
    main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
    main.actuator.biastype = mj.mjtBias.mjBIAS_AFFINE
    main.actuator.gainprm[0] =  kp
    main.actuator.biasprm[1] = -kp

    actuators =[
        {
            "name":"linear_act",
            "target":"linear"
        }
    ]

    for prop in actuators:
      self.spec.add_actuator(**prop)

In [ ]:
arm = LinearArm()
model = arm.spec.compile()
print_xml(arm.spec.to_xml())


Let use the LinearArm Component to create an open chain model of Hexapod. Later on, we will create a closed loop model using equality constraint. In addition we will use contact exclude to remove undesirable collsion detection.

In [ ]:
class HexapodOpenChain:
  def __init__(self,body_radius=0.1,arm_length=0.1):
    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody
    self.scene_option = mj.MjvOption()

    # material
    self.spec.add_material(name = "purple",rgba=[0.5, 0.0, 0.5,1])

    base_prop = {
      "body":{
        "name":"base"
      },
      "geom":{
        "size":[body_radius]*2+[body_radius/2],
        "type":mj.mjtGeom.mjGEOM_ELLIPSOID
      }
    }
    top_prop = {
        "body":{
            "name":"top",
            "pos":[-body_radius,0,arm_length*2]
        },
        "geom":{
        "size":[body_radius]*2+[body_radius/2],
        "type":mj.mjtGeom.mjGEOM_ELLIPSOID,
        "material":"purple"
        },
        "joint":{
            # "type":mj.mjtJoint.mjJNT_HINGE,
            "type":mj.mjtJoint.mjJNT_BALL,
            # "axis":[1,0,0],
            "pos":[body_radius,0,0]

        }

    }

    # classname


    # attaching arms to base
    self.base = self.model.add_body(**base_prop['body'])
    self.base.add_geom(**base_prop['geom'])

    self.arms = []
    self.num_arms = 3
    for i in range(self.num_arms):
      theta = 2 * i * np.pi / self.num_arms
      arm_pos = body_radius * np.array([np.cos(theta), np.sin(theta), 0])
      frame = self.base.add_frame(pos = arm_pos,euler = [0,0,theta])
      arm = LinearArm()
      body_frame = frame.attach_body(arm.model, '', '-' + str(i))
      self.arms.append(body_frame)

    # defining joint between
    # first arm and top
    first_arm_l1 = self.arms[0].bodies[0]
    first_arm_l2 = first_arm_l1.bodies[0]
    self.top = first_arm_l2.add_body(**top_prop['body'])
    self.top.add_geom(**top_prop['geom'])
    self.top.add_joint(**top_prop['joint'])

  def visulize_joints(self):
    self.scene_option.flags[mj.mjtVisFlag.mjVIS_JOINT] = True

  def visulize_contact(self):
    self.scene_option.flags[mj.mjtVisFlag.mjVIS_CONTACTPOINT] = True

In [ ]:
hexapod = HexapodOpenChain()
model = hexapod.spec.compile()
print_xml(hexapod.spec.to_xml())

In [ ]:
arena = Arena()
robot = HexapodOpenChain()
robot.visulize_joints()
robot.visulize_contact()
arena.add_fixed_asset(robot,[0,0,0.1])
model = arena.spec.compile()
random_state = np.random.RandomState(42)

actuators = [actuator.id for actuator in arena.spec.actuators]
# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 3    # (seconds)
framerate = 30  # (Hz)
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=robot.scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

Next we will add equality constraint between the top body and second and third linear arm to create a closed loop kinematic. We will also remove undesirable contacts.

In [ ]:
class Hexapod(HexapodOpenChain):
  def __init__(self,body_radius=0.1,arm_length=0.1):
    super().__init__(body_radius,arm_length)

    # adding equality constraint
    # between second and third arm
    # and top
    for i in range(1,self.num_arms):
      theta = 2 * i * np.pi / self.num_arms
      site_pos = body_radius * np.array([np.cos(theta), np.sin(theta), 0])
      top_site = self.top.add_site(pos=site_pos,name="top_site-"+str(i),euler=[0,0,theta])
      l2 = self.arms[i].bodies[0].bodies[0]
      l2_site = l2.sites[0]
      self.spec.add_equality(objtype=mj.mjtObj.mjOBJ_SITE,
                            name1=top_site.name,
                            name2=l2_site.name)

    # contact exclusion
    for i in range(self.num_arms):
      arm = self.arms[i]
      # arm links
      l1 = arm.bodies[0]
      l2 = l1.bodies[0]
      self.spec.add_exclude(bodyname1=self.base.name,bodyname2=l1.name)
      self.spec.add_exclude(bodyname1=self.base.name,bodyname2=l2.name)
      if i==0:
        continue
      self.spec.add_exclude(bodyname1=self.top.name,bodyname2=l2.name)

  def visulize_equality(self):
    self.scene_option.flags[mj.mjtVisFlag.mjVIS_CONSTRAINT] = True

In [ ]:
hexapod = Hexapod()
model = hexapod.spec.compile()
print_xml(hexapod.spec.to_xml())

In [ ]:
arena = Arena()
robot = Hexapod()
robot.visulize_joints()
robot.visulize_contact()
arena.add_fixed_asset(robot,[0,0,0.1])
model = arena.spec.compile()
random_state = np.random.RandomState(42)

actuators = [actuator.id for actuator in arena.spec.actuators]
# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 3    # (seconds)
framerate = 30  # (Hz)
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=robot.scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

### Tendons

Here we will have a look at spatial and fixed tendon.

In [ ]:
class ArmBase:
  def __init__(self,create_tree=True,num_bodies=3):
    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody
    self.scene_option = mj.MjvOption()

    # material
    self.spec.add_material(name = "yellow",rgba=[1,1,0,1])
    self.spec.add_material(name = "orange",rgba=[1.0, 0.5, 0.0, 1.0])

    # defaults
    main = self.spec.default()
    main.joint.axis = [0,1,0]

    length=0.2
    self.bodies = []

    for i in range(num_bodies):
      if i>0:
        pos = [0,0,(length+.03)]
      else:
        pos = [0]*3

      b={
        "body":{
          "name":"l"+str(i),
          "pos":pos
        },
        "geoms":[
          {
            "mass":0.001,
            "name":"g1_"+str(i),
            "type":mj.mjtGeom.mjGEOM_CYLINDER,
            "fromto":[0, .015, 0, 0, -.015, 0],
            "size":[.05,0,0],
            "material":"orange"
          },
          {
            "mass":0.001,
            "name":"g2_"+str(i),
            "material":"yellow",
            "type":mj.mjtGeom.mjGEOM_CAPSULE,
            "fromto":[0]*5+[length],
            "size":[0.02,0,0]
          }
        ],
        "joint":{
            "name":"j"+str(i),
            "range":[-1,1]
        }
      }
      self.bodies.append(b)

    if create_tree:
      self._create_tree()

  def _create_tree(self):
    body = self.model
    for prop in self.bodies:
      body = body.add_body(**prop['body'])
      for geom in prop['geoms']:
        body.add_geom(**geom)
      if "joint" in prop:
        body.add_joint(**prop['joint'])
      if  "sites" in prop:
        for site in prop["sites"]:
          body.add_site(**site)


In [ ]:
arm = ArmBase()
model = arm.spec.compile()
print_xml(arm.spec.to_xml())


In [ ]:
class ArmFixedTendonCoupling(ArmBase):
  def __init__(self):
    super().__init__()
    # Kinematic coupling: The passive joint’s motion is
    # limited or determined by the configuration and
    # movement of the active joint.

    tendon = "coupling_j1_j2"
    kinematic_coupling = [
      {
        "joint":"j1",
        "coef":1
      },
      {
        "joint":"j2",
        "coef":1
      }
    ]
    tendon = self.spec.add_tendon(name=tendon)
    for data in kinematic_coupling:
      tendon.wrap_joint(data["joint"],data["coef"])


In [ ]:
arm = ArmFixedTendonCoupling()
model = arm.spec.compile()
print_xml(arm.spec.to_xml())

In [ ]:
class ArmFixedTendon(ArmFixedTendonCoupling):
  def __init__(self):
    super().__init__()

    kp = 1
    # defaults
    main = self.spec.default()

    # position actuator defaults

    main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
    main.actuator.biastype = mj.mjtBias.mjBIAS_AFFINE
    main.actuator.forcerange = [-1,1]

    kp = 1
    # copying default values
    gainprm = main.actuator.gainprm[:]
    biasprm = main.actuator.biasprm[:]
    # setting kp
    gainprm[0] =  kp
    biasprm[1] = -kp

    t_actuator = {
        "name":"act_tendon",
        "target":"coupling_j1_j2",
        "ctrlrange":[-1,1],
        "gainprm":gainprm,
        "biasprm":biasprm,
        "trntype":mj.mjtTrn.mjTRN_TENDON
    }
    p_actuator = {
      "name":"act_j0",
      "target": "j0",
      "trntype":mj.mjtTrn.mjTRN_JOINT
    }

    # position actuator
    self.spec.add_actuator(**p_actuator)
    # tendon drive
    self.spec.add_actuator(**t_actuator)


In [ ]:
arm = ArmFixedTendon()
model = arm.spec.compile()
print_xml(arm.spec.to_xml())

In [ ]:
arena = Arena()
arm = ArmFixedTendon()
body = arena.add_fixed_asset(arm,pos=[0,0,0.15])
model = arena.spec.compile()
render(model)

In [ ]:
class ArmTendonsRouting(ArmBase):
  def __init__(self):
    super().__init__(create_tree=False,num_bodies=2)
    """
    First joint is position controlled
    Second joint
    will be go tendon driven
    """
    # adding sites to end of first body
    self.bodies[0]['sites'] = [
      {
        "name":"s1",
        "pos":[0.025,0,0.15]
      },
      {
        "name":"s2",
        "pos":[-0.025,0,0.15]
      },
    ]
    # adding sites to beginning of first body
    self.bodies[1]['sites'] = [
      {
        "name":"s3",
        "pos":[0.025,0,0.08]
      },
      {
        "name":"s4",
        "pos":[-0.025,0,0.08]
      },
      # these sites will be used for routing
      {
        "name":"sr1",
        "pos":[0.08,0,0]
      },
      {
        "name":"sr2",
        "pos":[-0.08,0,0]
      }
    ]
    # creating tree like structure of robot
    self._create_tree()

    # adding tendons to model
    tendons = [
      # right tendon
      {
        "name":"left",
        "data":[
          {
            "name":"s1"
          },
          {
            "name":"g1_1",
            "sidesite":"sr1"

          },
          {
            "name":"s3"
          }
        ]
      },
      # left tendon
      {
        "name":"right",
        "data":[
          {
            "name":"s2"
          },
          {
            "name":"g1_1",
            "sidesite":"sr2"

          },
          {
            "name":"s4"
          }
        ]
      },
    ]

    for prop in tendons:
      tendon = self.spec.add_tendon(name=prop["name"])
      for d in prop['data']:
        if not 'sidesite' in d:
          tendon.wrap_site(d['name'])
        else:
          tendon.wrap_geom(d['name'],d['sidesite'])


In [ ]:
arm = ArmTendonsRouting()
model = arm.spec.compile()
print_xml(arm.spec.to_xml())

In [ ]:
class ArmMuscle(ArmTendonsRouting):
    def __init__(self):
      super().__init__()

      # defaults
      main = self.spec.default()

      # muscle defaults
      main.actuator.trntype = mj.mjtTrn.mjTRN_TENDON
      main.actuator.dyntype  = mj.mjtDyn.mjDYN_MUSCLE
      main.actuator.gaintype = mj.mjtGain.mjGAIN_MUSCLE
      main.actuator.biastype = mj.mjtBias.mjBIAS_MUSCLE

      main.actuator.dynprm[0] = 0.01 # tau act
      main.actuator.dynprm[1] = 0.04 # tau deact

      # range[0], range [1], force, scale,
      # lmin, lmax, vmax, fpmax, fvmax
      main.actuator.gainprm[0:9] = [0.75,1.05,-1,200,
                                    0.5,1.6,1.5,1.3,1.2]

      # biasprm = gainprm
      main.actuator.biasprm = main.actuator.gainprm[:]

      actuators = [
        {
          "name":"act_left",
          "target":"left"
        },
        {
          "name":"act_right",
          "target":"right"
        },
      ]
      # muscle actuators
      for actuator in actuators:
        self.spec.add_actuator(**actuator)


In [ ]:
arm = ArmMuscle()
model = arm.spec.compile()
print_xml(arm.spec.to_xml())

In [ ]:
arena = Arena()
arm = ArmMuscle()
body = arena.add_fixed_asset(arm,pos=[0,0,0.15])
model = arena.spec.compile()
render(model)

### Mesh

In [ ]:
cf2_assets = 'mujoco_menagerie/bitcraze_crazyflie_2/assets/'

In [ ]:
@dataclasses.dataclass
class CrazyFlieData:
  asset_path:str
  materials = [
    {
        "name":"polished_plastic",
        "rgba":[0.631, 0.659, 0.678, 1]
    },
    {
        "name":"polished_gold",
        "rgba":[0.969, 0.878, 0.6, 1]
    },
    {
        "name":"medium_gloss_plastic",
        "rgba":[0.109, 0.184, 0.0, 1]
    },
    {
        "name":"propeller_plastic",
        "rgba":[0.792, 0.820, 0.933, 1]
    },
    {
        "name":"white",
        "rgba":[1]*4
    },
    {
        "name":"body_frame_plastic",
        "rgba":[0.102, 0.102, 0.102, 1]
    },
    {
        "name":"burnished_chrome",
        "rgba":[0.898, 0.898, 0.898, 1]
    }
  ]
  # meshes for visulizaition
  meshes_vis = {
    "prefix":"cf2_",
    "range":[0,6]
  }
  # meshes for collision
  meshes_col = {
      "prefix":"cf2_collision_",
      "range":[0,31]
  }
  # default
  geom_vis_default = {
      "type":mj.mjtGeom.mjGEOM_MESH,
      "group":2,
      "contype":0,
      "conaffinity":0
  }
  geom_col_default = {
      "type":mj.mjtGeom.mjGEOM_MESH,
      "group":3
  }
  # props

  body = {
      "name":"cf2",
      "pos": [0, 0, 1],
      # ipos and iquat can be removed since they are
      # the default values
      "ipos":[0,0,0],
      "iquat":[1,0,0,0],
      "mass":0.027,
      "inertia":[2.3951e-5, 2.3951e-5, 3.2347e-5],
      "explicitinertial":True

  }
  geoms_col = meshes_col
  geoms_vis = meshes_vis
  geom_vis_material = ["propeller_plastic", "medium_gloss_plastic",
    "polished_gold", "polished_plastic", "burnished_chrome",
    "body_frame_plastic","white"
  ]
  sites = [
      {
          "name":"imu",
          "group":5
      },
      {
          "name":"actuation",
          "group":5
      }
  ]
  sensors = [
       {
           "name":"body_gyro",
            "type":mj.mjtSensor.mjSENS_GYRO,
            "objtype":mj.mjtObj.mjOBJ_SITE,
            "objname":"imu"
        },
       {
           "name":"body_linacc",
            "type":mj.mjtSensor.mjSENS_ACCELEROMETER,
            "objtype":mj.mjtObj.mjOBJ_SITE,
            "objname":"imu"
        },
       {
           "name":"body_quat",
            "type":mj.mjtSensor.mjSENS_FRAMEQUAT,
            "objtype":mj.mjtObj.mjOBJ_SITE,
            "objname":"imu"
        }
  ],
  actuation = [
    {
      "ctrlrange":[0, 0.35],
      "gear":[0, 0, 1, 0, 0, 0],
      "target":"actuation",
      "name":"body_thrust"
    },
    {
      "ctrlrange":[-1, 1],
      "gear":[0, 0, 0, -0.00001, 0, 0],
      "target":"actuation",
      "name":"x_moment"
    },
    {
      "ctrlrange":[-1, 1],
      "gear":[0, 0, 0, 0, -0.00001, 0],
      "target":"actuation",
      "name":"y_moment"
    },
    {
      "ctrlrange":[-1, 1],
      "gear":[0, 0, 0, 0, 0, -0.00001],
      "target":"actuation",
      "name":"z_moment"
    }
  ]
  camera = {
      "name":"track",
      "pos":[-1, 0, .5],
      "xyaxes":[0, -1, 0, 1, 0, 2],
      "mode":mj.mjtCamLight.mjCAMLIGHT_TRACKCOM
  }

In [ ]:
class CrazyFlies:
  def __init__(self,data):
    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody

    # defaults
    main = self.spec.default()

    # motor actuation setting
    main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
    main.actuator.biastype = mj.mjtBias.mjBIAS_NONE
    main.actuator.dyntype  = mj.mjtDyn.mjDYN_NONE
    main.actuator.trntype = mj.mjtTrn.mjTRN_SITE

    # add material
    for m in data.materials:
      self.spec.add_material(**m)
    # add mesh
    num_col_mesh = data.meshes_col["range"][1]
    for i in range(num_col_mesh + 1):
      filename = data.meshes_col["prefix"]+str(i)
      self.spec.add_mesh(name=filename, file=data.asset_path+filename+'.obj')

    num_vis_mesh = data.meshes_vis["range"][1]
    for i in range(num_vis_mesh + 1):
      filename = data.meshes_vis["prefix"]+str(i)
      self.spec.add_mesh(name=filename, file=data.asset_path+filename+'.obj')

    # body
    body = self.model.add_body(**data.body)
    # camera
    body.add_camera(**data.camera)
    # geoms
    num_vis_geoms = data.geoms_vis["range"][1]
    for i in range(num_vis_geoms + 1):
      mesh = data.geoms_vis["prefix"]+str(i)
      geom = {
          "meshname":mesh,
          "material":data.geom_vis_material[i]
      }
      geom |=data.geom_vis_default
      body.add_geom(**geom)

    num_col_geoms = data.geoms_col["range"][1]
    for i in range(num_col_geoms + 1):
      mesh = data.geoms_col["prefix"]+str(i)
      geom = {
          "meshname":mesh
      }
      geom |= data.geom_col_default
      body.add_geom(**geom)
    # sites
    for site in data.sites:
      body.add_site(**site)

    # actuators
    for actuator in data.actuation:
      self.spec.add_actuator(**actuator)

In [ ]:
cf_data = CrazyFlieData(cf2_assets)
cf = CrazyFlies(cf_data)
model = cf.spec.compile()
print_xml(cf.spec.to_xml())

### Height field
Height field is used to represent uneaven train or mountains. There are many ways to generated a procedural terrain. Here we will use Perlin Noise to generate hight field:
https://www.youtube.com/watch?v=9x6NvGkxXhU

For more insperation have a look at:
https://www.youtube.com/watch?v=gsJHzBTPG0Y

We start by installing python package that helps with generation of Perlin Noise.


In [ ]:
# perlin noise
!pip install git+https://github.com/pvigier/perlin-numpy
from perlin_numpy import generate_perlin_noise_2d

In [ ]:
np.random.seed(0)
noise = generate_perlin_noise_2d((256, 256), (8, 8))
plt.imshow(noise, cmap='gray', interpolation='lanczos')
plt.colorbar()

In [ ]:
class ArenaHFileld:
  def __init__(self):
    self.spec = mj.MjSpec()
    self.model = self.spec.worldbody
    self.spec.compiler.degree = False
    self.scene_option = mj.MjvOption()


    # setting rendring window size
    # self.spec.visual.offwidth = 1280
    # self.spec.visual.offheight = 720


    # adding hfield
    noise = generate_perlin_noise_2d((256, 256), (8, 8))
    hfield = self.spec.add_hfield(name='hfield',
                                  size = [10,10,8,8],
                                  nrow = noise.shape[0],
                                  ncol = noise.shape[1],
                                  userdata=noise.flatten())

    # Make arena with textured floor.

    chequered = self.spec.add_texture(
        name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
        builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
        width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])

    grid = self.spec.add_material(
        name='chequered', texrepeat=[5, 5], reflectance=.2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
    self.model.add_geom(
        type=mj.mjtGeom.mjGEOM_HFIELD, material='chequered',hfieldname='hfield')

    for x in [-2, 2]:
      self.model.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])



  def add_tracking_camera(self,body,name="eye",pos=[0,0,1],euler=[0,0,0]):
    # camera
    self.model.add_camera(name=name,
                          mode =mj.mjtCamLight.mjCAMLIGHT_TRACK ,
                          targetbody=body,
                          pos= pos,
                          euler=euler)
    return name

  def visulize_camera(self):
    self.scene_option.flags[mj.mjtVisFlag.mjVIS_CAMERA] = True

  def add_movable_asset(self,asset,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(asset.model,  prefix,'')
    body.add_freejoint()
    return body

  def add_fixed_asset(self,asset,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(asset.model,  prefix,'')
    return body

  def remove_asset(self,body):
    self.spec.detach_body(body)

In [ ]:
arena = ArenaHFileld()
model = arena.spec.compile()
render(model)

In [ ]:
arena = ArenaHFileld()
robot = MultiLegRobot()
arena.add_movable_asset(robot,[0,0,8])
camera = arena.add_tracking_camera("_world",euler=[0,0,0],pos=[0,0,10])
arena.visulize_camera()
model = arena.spec.compile()
random_state = np.random.RandomState(42)


# print(arena.spec.to_xml())

actuators = [actuator.id for actuator in arena.spec.actuators]
# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 5    # (seconds)
framerate = 30  # (Hz)
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model,width=640,height=480) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      # renderer.update_scene(data,scene_option=arena.scene_option)
      renderer.update_scene(data,camera=camera)

      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

### Procedural Mesh Generation
Mujoco has the functionality to create a mesh given verticies. These verities can be result of Prodcedural mesh generation. A person can use libraires like cad query :https://cadquery.readthedocs.io/en/latest/
to generate meshes and then construct them with mujoco. Here we will not demonstrate procedural mesh generation. However we are going to see how the resulting verticies can be used in mujoco.

Low poly rock:
https://www.youtube.com/watch?v=1H2YaRPfpdM

In [ ]:
# Texture
!git clone https://github.com/mohammad200h/dirt.git

!cd dirt && ls

In [ ]:
textures = [
    "./dirt/black-stone.png",
    "./dirt/stone-surface-background.png",
    "./dirt/stone-texture-background.png",
    "./dirt/top-view-soil.png",
]

data = {
    "faces": [[0, 9, 10], [8, 3, 28], [4, 13, 18], [2, 19, 22], [1, 20, 27],
 [6, 25, 52], [12, 7, 34], [17, 11, 36], [23, 16, 41], [29, 24, 46], [5, 51, 31],
  [14, 30, 35], [15, 37, 42], [21, 43, 48], [26, 47, 50], [32, 54, 55], [39, 33, 56],
   [44, 38, 57], [45, 40, 58], [53, 49, 59]],
    "vertices": [[0.0, -0.01, 0.0], [0.0, -0.01, 0.0], [0.0, -0.01, 0.0], [0.0, -0.01, 0.0],
                 [0.0, -0.01, 0.0], [0.007236, -0.004472, 0.005257], [0.007236, -0.004472, 0.005257],
                 [0.007236, -0.004472, 0.005257], [0.007236, -0.004472, 0.005257], [0.007236, -0.004472, 0.005257],
                 [-0.002764, -0.004472, 0.008506], [-0.002764, -0.004472, 0.008506], [-0.002764, -0.004472, 0.008506],
                 [-0.002764, -0.004472, 0.008506], [-0.002764, -0.004472, 0.008506], [-0.008944, -0.004472, 0.0],
                 [-0.008944, -0.004472, 0.0], [-0.008944, -0.004472, 0.0], [-0.008944, -0.004472, 0.0],
                 [-0.008944, -0.004472, 0.0], [-0.002764, -0.004472, -0.008506], [-0.002764, -0.004472, -0.008506],
                 [-0.002764, -0.004472, -0.008506], [-0.002764, -0.004472, -0.008506], [-0.002764, -0.004472, -0.008506],
                 [0.007236, -0.004472, -0.005257], [0.007236, -0.004472, -0.005257], [0.007236, -0.004472, -0.005257],
                 [0.007236, -0.004472, -0.005257], [0.007236, -0.004472, -0.005257], [0.002764, 0.004472, 0.008506],
                 [0.002764, 0.004472, 0.008506], [0.002764, 0.004472, 0.008506], [0.002764, 0.004472, 0.008506],
                 [0.002764, 0.004472, 0.008506], [-0.007236, 0.004472, 0.005257], [-0.007236, 0.004472, 0.005257],
                 [-0.007236, 0.004472, 0.005257], [-0.007236, 0.004472, 0.005257], [-0.007236, 0.004472, 0.005257],
                 [-0.007236, 0.004472, -0.005257], [-0.007236, 0.004472, -0.005257], [-0.007236, 0.004472, -0.005257],
                 [-0.007236, 0.004472, -0.005257], [-0.007236, 0.004472, -0.005257], [0.002764, 0.004472, -0.008506],
                 [0.002764, 0.004472, -0.008506], [0.002764, 0.004472, -0.008506], [0.002764, 0.004472, -0.008506],
                 [0.002764, 0.004472, -0.008506], [0.008944, 0.004472, 0.0], [0.008944, 0.004472, 0.0],
                 [0.008944, 0.004472, 0.0], [0.008944, 0.004472, 0.0], [0.008944, 0.004472, 0.0],
                 [0.0, 0.01, 0.0], [0.0, 0.01, 0.0], [0.0, 0.01, 0.0], [0.0, 0.01, 0.0], [0.0, 0.01, 0.0]]
}

class Rock:
  def __init__(self,data,textures:typing.List[str],name="rock"):
    self.spec = mj.MjSpec()
    self.model = self.spec.worldbody
    self.spec.compiler.degree = False


    main = self.spec.default()
    main.mesh.scale = [random.random()*20]*3

    self.spec.add_texture(
        name="rock", type=mj.mjtTexture.mjTEXTURE_2D,
        file = random.choice(textures),
        width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
    self.spec.add_material(
        name='rock',  reflectance=.2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'rock'
    # defaults
    main = self.spec.default()
    main.geom.type = mj.mjtGeom.mjGEOM_MESH

    self.spec.add_mesh(
        name = name,
        uservert = np.array(data["vertices"]).flatten() ,
        userface = np.array(data["faces"]).flatten()
    )
    self.model.add_geom(meshname = name , material = "rock")

In [ ]:
arena = Arena()

for i in range(10):
  name = "rock_"+str(i)
  asset = Rock(data,textures,name=name)
  arena.add_fixed_asset(asset,[0.1*i,0,0.11],prefix=name)

model = arena.spec.compile()
render(model)

### Rocky Situation
Here we will create a scene using every model we have created.Then we will add more obstacles to the scene by droping cubes eveywhere. We will start creating an arena using the Hfield. The quadroped is used to transfer things from location A to B. To make sure things don't fall off we attach the Hexopaad to the top of quadroped so that the load can be balanced. Finally when the quadroped reaches point B, an arm picks up the object.

https://github.com/google-deepmind/mujoco/blob/main/python/mujoco/renderer_test.py


### Transport

In [ ]:
class Transport:
  def __init__(self):
    quadroped = MultiLegRobot()
    hexopad = Hexapod()

    body = hexopad.top

    arm_frames = [
        {
            "name":"left",
            "pos":[0.1,0,0],
            "euler":[0,1.2,0],
            "arm":ArmFixedTendon().model
        },
        {
            "name":"right",
            "pos":[-0.1,0,0],
            "euler":[0,-1.2,0],
            "arm":ArmMuscle().model
        }
    ]

    for prop in arm_frames:
      frame = body.add_frame(pos = prop['pos'],euler=prop['euler'])
      frame.attach_body(prop['arm'],"","_"+prop["name"])


    frame = quadroped.model.add_frame(pos=[0,0,0])
    frame.attach_body(hexopad.model, '','_')

    self.spec = quadroped.spec
    self.model = quadroped.model



In [ ]:
arena = Arena()
robot = Transport()
arena.add_fixed_asset(robot,[0,0,0.15])
model = arena.spec.compile()
render(model)

In [ ]:
def circle_coordinates(radius, angle):
    x = radius * math.cos(angle)
    y = radius * math.sin(angle)
    return x, y


In [ ]:
arena = Arena()
robot = Transport()

# rocks
radius = 1
for i,angle in enumerate(np.linspace(-3.14,3.14,10)):
  x,y = circle_coordinates(radius/2,angle)
  name = "inner_rock_"+str(i)
  asset = Rock(data,textures,name=name)
  arena.add_movable_asset(asset,[x,y,0.11],prefix=name)

for i,angle in enumerate(np.linspace(-3.14,3.14,20)):
  x,y = circle_coordinates(radius,angle)
  name = "outer_rock_"+str(i)
  asset = Rock(data,textures,name=name)
  arena.add_movable_asset(asset,[x,y,0.11],prefix=name)



arena.add_movable_asset(robot,[0,0,3])
camera = arena.add_tracking_camera("_world",euler=[0,0,0],pos=[0,0,3])
arena.visulize_camera()
model = arena.spec.compile()
random_state = np.random.RandomState(42)


# print(arena.spec.to_xml())

actuators = [actuator.id for actuator in arena.spec.actuators]
# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 10    # (seconds)
framerate = 30  # (Hz)
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model,width=640,height=480) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      # renderer.update_scene(data,scene_option=arena.scene_option)
      renderer.update_scene(data,camera=camera)

      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)